In [1]:
from keras import models
from keras import layers

model = models.Sequential()
# 첫 번째 차원이 784인 2D 텐서만 입력으로 받는 층, 첫 번째 차원의 크기가 32로 변환된 텐서를 출력.
model.add(layers.Dense(32, input_shape=(784,)))
# 케라스에서는 모델에 추가된 층을 자동으로 상위 층의 크기에 맞추어 줌.
model.add(layers.Dense(10))

Using TensorFlow backend.


In [2]:
def read_data(filename):
    with open(filename, 'rt', encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        # txt 파일의 헤더(id document label)는 제외하기
        data = data[1:]
    return data

train_data = read_data('./ratings_train.txt')
test_data = read_data('./ratings_test.txt')

In [3]:
from konlpy.tag import Okt

okt = Okt()

In [4]:
import json
import os
from pprint import pprint

def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

if os.path.isfile('train_docs.json'):
    with open('train_docs.json', 'a', encoding='utf-8') as f:
        train_docs = json.load(f)
    with open('test_docs.json', 'a', encoding='utf-8') as f:
        test_docs = json.load(f)
else:
    train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
    test_docs = [(tokenize(row[1]), row[2]) for row in test_data]
    # JSON 파일로 저장
    with open('train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")

# 예쁘게(?) 출력하기 위해서 pprint 라이브러리 사용
pprint(train_docs[0])

(['아/Exclamation',
  '더빙/Noun',
  '../Punctuation',
  '진짜/Noun',
  '짜증나다/Adjective',
  '목소리/Noun'],
 '0')


In [5]:
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

2159921


In [6]:
import nltk
text = nltk.Text(tokens, name='NMSC')

In [7]:
# 시간이 꽤 걸립니다! 시간을 절약하고 싶으면 most_common의 매개변수를 줄여보세요.
selected_words = [f[0] for f in text.vocab().most_common(1000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

In [8]:
import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [9]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(1000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 150000 samples
Epoch 1/10
150000/150000 [==============================] - 1s 10us/sample - loss: 0.4341 - binary_accuracy: 0.8014
Epoch 2/10
150000/150000 [==============================] - 2s 12us/sample - loss: 0.3827 - binary_accuracy: 0.8229
Epoch 3/10
150000/150000 [==============================] - 1s 10us/sample - loss: 0.3672 - binary_accuracy: 0.8320
Epoch 4/10
150000/150000 [==============================] - 2s 13us/sample - loss: 0.3541 - binary_accuracy: 0.8399
Epoch 5/10
150000/150000 [==============================] - 2s 10us/sample - loss: 0.3420 - binary_accuracy: 0.8466
Epoch 6/10
150000/150000 [==============================] - 1s 9us/sample - loss: 0.3303 - binary_accuracy: 0.8531
Epoch 7/10
150000/150000 [==============================] - 1s 9us/sample - loss: 0.3182 - binary_accuracy

In [13]:
#분유 제조사 목록
milkName = ['갈리아', '거버', '남양유업', '네슬레', '네이쳐스원', '노발락', '뉴트리시아', '뢰벤짠', '매일유업', '메이지', '밀라산', '밀루파', 
            '벨라미스오가닉', '아스펜', '아이배냇', '아이엠뉴질랜드', '애보트', '얼스베스트', '엔파밀', '오브맘', '와코도', '일동후디스', '카브리타',
            '커클랜드', '퇴퍼', '파스퇴르', '페디아슈어', '퓨어랜드', '후마나', 'HOLLE', 'Hipp', 'LG생활건강']

In [14]:
slowfood = ['남양유업',  '네슬레', '래퍼티스가든', '맘스쌀과자', '매일유업', '미즈앤코', '베베쿡', '비치넛', '에바토', '일동후디스', '파스퇴르', 
            'LG생활건강', 'SafrunatS.L', '루솔', '뽀뽀뜨', '아이배냇', '얼스베스트', '엘빈즈', '오가닉맘', '짱죽', '팜투베이비', '푸드케어', '풀무원']

In [15]:
gijugi = [ '겐키', '군', '깨끗한나라', '나비잠', '네띠', '네이쳐러브메레', '네추럴블라썸', '네츄라오가닉', '대디베이비', '드라이퍼스', '로맘스', 
          '로한', '리베로', '리보쓰', '리에또', '마르마이플러스', '마미포코', '맘스네이처', '메리즈', '메리트산업', '메이커키즈', '모나리자', 
          '모모래빗', '모이몰른', '몰텍스', '무니', '무루', '무미', '밍크뮤', '바바랩', '밤보', '밤부베베', '백양산업', '범킨스', '베베린도',
           '베베몬', '베베원', '베이비리', '베이비스토리', '베이비앙', '베이비앤아이', '베이비오가닉', '베피스', '벤스랜드', '보솜이', 
          '블루독베이비', '서림', '세븐스제너레이션', '세한', '쇼콜라', '수안산업', '슈퍼대디', '신성', '쌍용C&B', '아가똥', '아가짱', '아이두레',
           '아이씨프로젝트', '아이에이커머스', '아이엠트루', '아이웰', '아이티씨', '암웨이', '압소바', '앙블랑', '애플크럼비', '에뜨와', 
          '에이메르네이쳐', '에코튜드네이처', '에프랑', '엠케이', '영림B&A', '오가닉맘', '오로라', '오보소', '오아이비', '올러브', '유트러스', 
          '은홍섬유', '이마트', '이엔컴퍼니', '이오스트', '일동아이엠씨', '조이로이', '주치니', '주타노다이퍼', '참사랑', '천싸요', '첨이첨이', 
          '치쿨', '캉가케어', '커클랜드', '케이맘', '코트니', '코튼리퍼블릭', '쿠쉬스', '쿠잉', '큐비앤맘', '클로즈', '키즈디조이', '킨도', '킷앤킨',
           '토디앙', '파라솔', '파코라반베이비', '팸퍼스', '페넬로페', '펠트친구', '포그니', '포유', '폴라리스', '폴맘', '피노', '핑고', '하기스', 
          '하이가', '한빛산업', '행키베베', '헤인셀레스티얼', '현대BABY산업', '홈플러스', 'BB DISTRIBE', 'GBSTYLE', 'GERBER', 'MK']

In [16]:
babyCar = [ '구름과환경', '구비', '굿베이비', '그라코', '그린탐', '깜', '나단스', '뉴나', '다이치', '델타', '도렐', '동방레포츠', '라스깔라', '레카로', '로드스타', '리안', 
           '리첼', '리틀타익스', '린덴', '마루스', '마리꼬베베', '마마루', '마마스앤파파스', '마이크라라이트', '맥시코시', '맥클라렌', '몽트뢰', '무무브', '무치', '미마', 
           '바론', '밤비노', '베이비몬스터', '베이비스타일', '베이비시티', '베이비싱', '베이비젠', '베이비조거', '베이비캠프', '베이비트랜드', '베이비프라임', '베이비홈', 
           '보브코리아', '부가부', '브라이택스', '브레비', '뻬그뻬레고', '샤오미', '세기산업', '세이프티퍼스트', '세피앙', '스퀴즈', '스토케', '스트롤에어', '실버크로스', 
           '실버팍스', '싸이벡스', '써니러브', '아가방앤컴퍼니', '아넥스', '아발론', '아이빌리브', '아이사랑', '아이엠베이비', '아이캔디', '아이쿠', '아프리카', '알로앤루', 
           '알퐁소', '앙팡스', '어파베이비', '에그', '에르고베이비', '에스디스피드', '에어보스', '에이블트레이딩', '엔픽스', '엘레갈로', '엘레니어', '엠버', '예떼', '오르빗베이비', 
           '오타브', '와이업', '와이케이비앤씨', '요야', '유니원', '이바겐', '이븐플로', '이지워커', '인스텝', '잉글라차','잉글레시나', '제로', '제인', '조이', '조이파트너스', 
           '조코', '줄즈', '중모토이플러스', '쥬비', '지비', '쭈쭈베베', '차일드홈', '치코', '카펠라', '코뮌코리아', '코사토', '코엔코', '콜크래프트', '콤비', '콩코드', '퀴니', 
           '큐터스', '키도포타머스', '키디', '키즈엠브레이스', '킨더웨건', '킨즈', '테크넘', '토이파파', '톨스토이', '톰비', '툴레', '파파앤코', '팔리', '페도라', '포브', '프시케', 
           '필앤테즈', '하이브리드', '한스', '한일레인보우', '해님', '해밀턴', '해피랜드F&C', '햇님토이', '헨티', 'ABC', 'GB', 'MEIBAO', 'SEEC', 'VOVO', 'belecoo']

In [17]:
import csv

for j in milkName:

    feedback_list=[]

    with open(f'../1. merge/1. 분유/{j}_merge.csv', 'r', encoding='utf-8') as f:
        csv_reader = csv.reader(f, delimiter = '\n')
        for row in csv_reader:
            feedback_list.append(row)

    feedback_list = sum(feedback_list, [])

    def predict_pos_neg_fix(review):
        token = tokenize(review)
        tf = term_frequency(token)
        data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
        score = float(model.predict(data))
        if(score > 0.5):
            positive001.append("[{}], 긍정 텍스트 확률 : {:.2f}%".format(review, score * 100))
        else:
            negative001.append("[{}], 부정 텍스트 확률 : {:.2f}%".format(review, (1 - score) * 100))

    positive001 = []
    negative001 = []

    for i in range(0, len(feedback_list)):
        predict_pos_neg_fix(str(feedback_list[i]))

    import pandas as pd
    evaluate_pos001 = pd.DataFrame(positive001)
    evaluate_pos001.to_csv(f"./1. 분유/{j}_positive.csv", header = False, index = False)
    evaluate_neg001 = pd.DataFrame(negative001)
    evaluate_neg001.to_csv(f"./1. 분유/{j}_negative.csv", header = False, index = False)

In [18]:
for j in slowfood:

    feedback_list=[]

    with open(f'../1. merge/2. 이유식/{j}_merge.csv', 'r', encoding='utf-8') as f:
        csv_reader = csv.reader(f, delimiter = '\n')
        for row in csv_reader:
            feedback_list.append(row)

    feedback_list = sum(feedback_list, [])

    def predict_pos_neg_fix(review):
        token = tokenize(review)
        tf = term_frequency(token)
        data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
        score = float(model.predict(data))
        if(score > 0.5):
            positive001.append("[{}], 긍정 텍스트 확률 : {:.2f}%".format(review, score * 100))
        else:
            negative001.append("[{}], 부정 텍스트 확률 : {:.2f}%".format(review, (1 - score) * 100))

    positive001 = []
    negative001 = []

    for i in range(0, len(feedback_list)):
        predict_pos_neg_fix(str(feedback_list[i]))

    import pandas as pd
    evaluate_pos001 = pd.DataFrame(positive001)
    evaluate_pos001.to_csv(f"./2. 이유식/{j}_positive.csv", header = False, index = False)
    evaluate_neg001 = pd.DataFrame(negative001)
    evaluate_neg001.to_csv(f"./2. 이유식/{j}_negative.csv", header = False, index = False)

In [19]:
for j in gijugi:

    feedback_list=[]

    with open(f'../1. merge/3. 기저귀/{j}_merge.csv', 'r', encoding='utf-8') as f:
        csv_reader = csv.reader(f, delimiter = '\n')
        for row in csv_reader:
            feedback_list.append(row)

    feedback_list = sum(feedback_list, [])

    def predict_pos_neg_fix(review):
        token = tokenize(review)
        tf = term_frequency(token)
        data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
        score = float(model.predict(data))
        if(score > 0.5):
            positive001.append("[{}], 긍정 텍스트 확률 : {:.2f}%".format(review, score * 100))
        else:
            negative001.append("[{}], 부정 텍스트 확률 : {:.2f}%".format(review, (1 - score) * 100))

    positive001 = []
    negative001 = []

    for i in range(0, len(feedback_list)):
        predict_pos_neg_fix(str(feedback_list[i]))

    import pandas as pd
    evaluate_pos001 = pd.DataFrame(positive001)
    evaluate_pos001.to_csv(f"./3. 기저귀/{j}_positive.csv", header = False, index = False)
    evaluate_neg001 = pd.DataFrame(negative001)
    evaluate_neg001.to_csv(f"./3. 기저귀/{j}_negative.csv", header = False, index = False)

In [20]:
for j in babyCar:

    feedback_list=[]

    with open(f'../1. merge/4. 유모차/{j}_merge.csv', 'r', encoding='utf-8') as f:
        csv_reader = csv.reader(f, delimiter = '\n')
        for row in csv_reader:
            feedback_list.append(row)

    feedback_list = sum(feedback_list, [])

    def predict_pos_neg_fix(review):
        token = tokenize(review)
        tf = term_frequency(token)
        data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
        score = float(model.predict(data))
        if(score > 0.5):
            positive001.append("[{}], 긍정 텍스트 확률 : {:.2f}%".format(review, score * 100))
        else:
            negative001.append("[{}], 부정 텍스트 확률 : {:.2f}%".format(review, (1 - score) * 100))

    positive001 = []
    negative001 = []

    for i in range(0, len(feedback_list)):
        predict_pos_neg_fix(str(feedback_list[i]))

    import pandas as pd
    evaluate_pos001 = pd.DataFrame(positive001)
    evaluate_pos001.to_csv(f"./4. 유모차/{j}_positive.csv", header = False, index = False)
    evaluate_neg001 = pd.DataFrame(negative001)
    evaluate_neg001.to_csv(f"./4. 유모차/{j}_negative.csv", header = False, index = False)